In [17]:
import requests
import altair as alt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import requests
import numpy as np
import datetime
from chessdotcom import get_player_game_archives

In [18]:
player = 'grzegorzgatkowski'
data = ['https://api.chess.com/pub/player/grzegorzgatkowski/games/2022/10']
response = requests.get(data[0]).json()['games']

In [19]:
def get_games(monthly_games):
    all_months = pd.DataFrame()
    for url in monthly_games:
        if url[-7:-3]=='2022':
                response = requests.get(url).json()['games']
                month = pd.json_normalize(response)
                all_months=pd.concat([all_months, month])
    return all_months

In [ ]:
%timeit b = get_games(data)

In [ ]:
def get_years_list(player_name):
    try:
        response = requests.get(f"https://api.chess.com/pub/player/{player_name}/games/archives")
        response.raise_for_status()
    except requests.RequestException:
        return None

    try:
        year = response.json()["archives"][0][-7:-3]
        first_year = int(year)
        list_year = [*range(first_year,2023)]
        str_list = [str(x) for x in list_year]
        
        return str_list
    except (KeyError, TypeError, ValueError):
        return None   

In [ ]:
a = get_years_list('grzegorzgatkowski')

In [ ]:
print(a)

In [26]:
all_months = get_games(data)
all_months.end_time = pd.to_datetime(all_months.end_time,unit='s')
all_months[player+"'s rating"] = np.where(all_months['white.username']==player,all_months['white.rating'],all_months['black.rating'])

In [27]:
all_months["opponent's rating"] = np.where(all_months['white.username']!=player,all_months['white.rating'],all_months['black.rating'])
all_months[player+" accuracy"] = np.where(all_months['white.username']==player,all_months['accuracies.white'],all_months['accuracies.black'])
all_months["Opponent accuracy"] = np.where(all_months['white.username']!=player,all_months['accuracies.white'],all_months['accuracies.black'])

KeyError: 'accuracies.white'

In [ ]:
all_months.head(1)

In [ ]:

df = all_months.groupby('time_class').resample('M', on = 'end_time')[[player+"'s rating"]].mean().reset_index()
df_wide = df.pivot("end_time", "time_class", player+"'s rating")

df_wide = df_wide.fillna(method="ffill")
df_wide.head()